In [51]:
# collectRI = pd.read_csv("https://github.com/pablormier/omnipath-static/raw/main/op/collectri-26.09.2023.zip")
# collectRI.to_csv(f'resources/grn_models/d0_hvgs/collectri.csv')
# collectRI.to_csv(f'resources/grn_models/default/collectri.csv')

In [24]:
!ls resources/grn-benchmark//

multiomics_atac_d0.h5ad  multiomics_rna_d0.h5ad      multiomics_rna.rds
multiomics_atac.h5ad	 multiomics_rna_d0_hvg.h5ad  perturbation_data.h5ad
multiomics_atac.rds	 multiomics_rna.h5ad
multiomics_rna_0.h5ad	 multiomics_rna_hvg.h5ad


# Sync

In [53]:
!aws s3 sync resources/grn-benchmark s3://openproblems-data/resources/grn/grn-benchmark --delete
!aws s3 sync resources/grn_models/ s3://openproblems-data/resources/grn/grn_models --delete
!aws s3 sync resources/prior/ s3://openproblems-data/resources/grn/prior --delete
!aws s3 sync resources/supplementary/ s3://openproblems-data/resources/grn/supplementary --delete
!aws s3 sync resources/results/ s3://openproblems-data/resources/grn/results --delete

upload: resources/grn_models/default/negative_control.csv to s3://openproblems-data/resources/grn/grn_models/default/negative_control.csv
upload: resources/grn_models/d0_hvgs/negative_control.csv to s3://openproblems-data/resources/grn/grn_models/d0_hvgs/negative_control.csv
upload: resources/results/d0_hvgs/trace.txt to s3://openproblems-data/resources/grn/results/d0_hvgs/trace.txt
delete: s3://openproblems-data/resources/grn/results/d0_hvgs_baseline/trace.txt


# Import

In [12]:
import yaml
import pandas as pd
import anndata as ad 
import numpy as np
import scanpy as sc 
from src.exp_analysis.helper import plot_cumulative_density
import sys 
sys.path.append('../')
from grn_benchmark.src.commons import surragate_names
def extract_data(data, reg='reg1', dataset_id='scgen_pearson'):
    i = 0
    for entry in data:
        if entry['dataset_id']!=dataset_id:
            continue
        try:
            rg, method_id = entry['method_id'].split('-')
        except:
            rg, method_id, _ = entry['method_id'].split('-')
        if rg != reg:
            continue
        dataset_id = entry['dataset_id']
        metric_ids = entry['metric_ids']
        metric_values = entry['metric_values']
        
        df = pd.DataFrame([metric_values], index=[method_id], columns=metric_ids)
        if i==0:
            df_reg = df
        else:
            df_reg = pd.concat([df_reg, df], axis=0)
        i+=1
    return df_reg
def process_data(RUN_ID, models_all=None):
    !aws s3 sync s3://openproblems-data/resources/grn/results/{RUN_ID} resources/results/{RUN_ID} 
    base_folder = f'resources/results/{RUN_ID}'
    result_file = f'{base_folder}/scores.yaml'
        

    with open(result_file, 'r') as file:
        data = yaml.safe_load(file)
    
    
    if models_all is None:
        df_reg1 = extract_data(data, reg='reg1').drop(columns=['Mean'])
        df_reg2 = extract_data(data, reg='reg2').drop(columns=['Mean'])
    else:
        df_reg1 = extract_data(data, reg='reg1').reindex(models_all).drop(columns=['Mean'])
        df_reg2 = extract_data(data, reg='reg2').reindex(models_all).drop(columns=['Mean'])
    # df_all = pd.concat([df_reg1, df_reg2], axis=1).fillna(0)
    # df_all_n = (df_all-df_all.min(axis=0))/(df_all.max(axis=0)-df_all.min(axis=0))
    # df_all['Rank'] = df_all_n.mean(axis=1).rank(ascending=False).astype(int)
    df_all = pd.concat([df_reg1, df_reg2], axis=1)
    return df_all

# Marco data 

In [2]:
# !ls resources_local/mccalla_extended/

In [3]:
# import subprocess
# import anndata as ad 
# import pandas as pd
# import numpy as np
# for cell_type in ['zhao', 'shalek', 'han', 'jackson']:
#     adata = ad.read_h5ad(f'resources_local/mccalla_extended/{cell_type}.h5ad')
#     adata.layers['norm'] = adata.X
#     adata.obs['cell_type'] = 'onecelltype'
#     adata.write(f'resources_local/mccalla_extended/{cell_type}.h5ad')
#     subsample = min([10000, len(adata)])
#     for GT in ['KDunion', 'chipunion', 'chipunion_KDUnion_intersect']:
#         GT_df = pd.read_csv(f'resources_local/mccalla_extended/{cell_type}_{GT}.csv')
#         gene_overlap = np.intersect1d(adata.var_names, GT_df.target.unique()).shape
#         print(f"{cell_type}-{GT}. adata shape: {adata.shape}, GT size: {GT_df.shape}, Gene overlap: {gene_overlap}")
#         command = f"viash run src/metrics/regression_1/config.vsh.yaml -- --perturbation_data resources_local/mccalla_extended/{cell_type}.h5ad --prediction resources_local/mccalla_extended/{cell_type}_{GT}.csv --layer norm --subsample {subsample} --apply_tf false --tf_all resources/prior/tf_all.csv --max_n_links -1 --verbose 1 --score output/{cell_type}_{GT}.h5ad"
#         subprocess.run(command, shell=True, check=True)

# d0_hvgs

In [ ]:
methods = [ 'collectri', 'negative_control', 'positive_control', 'pearson_corr', 'pearson_causal',  'portia', 'ppcor', 'genie3', 'grnboost2', 'scenic', 'scglue', 'celloracle']

In [ ]:
# consensus: run this after updating method list
# !python src/metrics/regression_2/consensus/script.py #inside the method names and dir


## GB

In [19]:
!sbatch scripts/sbatch/calculate_scores.sh #includes both reg1 and 2. #inside the script, set the reg_type, read and write dirs, and methods

Submitted batch job 7746169


## Scores

In [16]:
df_scores = pd.read_csv('resources/results/scores/ridge.csv', index_col=0)
df_scores.style.background_gradient()

,S1,S2,static-theta-0.0,static-theta-0.5
collectri,-0.100238,-0.211182,0.489316,0.514896
negative_control,-0.043795,-0.045561,0.459080,0.505002
positive_control,0.489147,0.677155,0.655407,0.574608
pearson_corr,0.238664,0.514612,0.529502,0.524232
pearson_causal,0.355256,0.578753,0.741328,0.560490
portia,0.148941,0.227248,0.451256,0.518048
ppcor,0.022846,0.094107,0.396680,0.509874
genie3,0.372641,0.490357,0.754073,0.576580
grnboost2,0.381032,0.459860,0.781852,0.609075
scenic,0.147553,0.214694,0.600839,0.574294


## Format resourcs used

### local runs

In [6]:
# extract trace_seqerafor local jobs
job_ids = {
    'portia': 7744548,
    'grnboost2': 7742249,
    'scenic': 7742283,
    'genie3': 7742285,
    'ppcor': 7742364,
    'scglue': 7742343,
}
import subprocess
import pandas as pd
import io

def get_sacct_data(job_id):
    command = f'sacct -j {job_id} --format=JobID,JobName,AllocCPUS,Elapsed,State,MaxRSS,MaxVMSize'
    output = subprocess.check_output(command, shell=True).decode()
    
    # Load the output into a DataFrame
    df = pd.read_csv(io.StringIO(output), delim_whitespace=True)
    df = df.iloc[[2]]
    return df

for i, (name, job_id) in enumerate(job_ids.items()):
    df = get_sacct_data(job_id)
    df.index = [name]
    if i==0:
        df_local = df
    else:
        df_local = pd.concat([df_local, df], axis=0)
def elapsed_to_hours(elapsed_str):
    h, m, s = map(int, elapsed_str.split(':'))
    return h + m / 60 + s / 3600
# Remove 'K' and convert to integers
df_local['MaxRSS'] = df_local['MaxRSS'].str.replace('K', '').astype(int)
df_local['MaxVMSize'] = df_local['MaxVMSize'].str.replace('K', '').astype(int)
df_local['Elapsed'] = df_local['Elapsed'].apply(lambda x: (elapsed_to_hours(x)))


# Convert MaxRSS and MaxVMSize from KB to GB
df_local['MaxRSS'] = df_local['MaxRSS'] / (1024 ** 2)  # Convert KB to GB
df_local['MaxVMSize'] = df_local['MaxVMSize'] / (1024 ** 2)  # Convert KB to GB
df_local

/vol/tmp/users/jnourisa/ipykernel_1858346/2555768217.py:19: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(io.StringIO(output), delim_whitespace=True)
/vol/tmp/users/jnourisa/ipykernel_1858346/2555768217.py:19: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(io.StringIO(output), delim_whitespace=True)
/vol/tmp/users/jnourisa/ipykernel_1858346/2555768217.py:19: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(io.StringIO(output), delim_whitespace=True)
/vol/tmp/users/jnourisa/ipykernel_1858346/2555768217.py:19: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  d

,JobID,JobName,AllocCPUS,Elapsed,State,MaxRSS,MaxVMSize
portia,7742241.bat+,batch,20,0.110556,COMPLETED,46.943497,47.474384
grnboost2,7742249.bat+,batch,20,1.568056,COMPLETED,3.067471,3.563801
scenic,7742283.bat+,batch,20,1.908056,COMPLETED,30.356461,32.573463
genie3,7742285.bat+,batch,20,16.682500,COMPLETED,13.105103,13.563530
ppcor,7742364.bat+,batch,20,0.556667,COMPLETED,3.909119,4.283978
scglue,7742343.bat+,batch,20,4.380278,FAILED,29.917423,35.933720


### sequra runs: multiple runs

In [7]:

def process_trace(trace):
    trace['model'] = pd.DataFrame(trace.name.str.split(':').to_list())[3] #TODO: number 3 might be different  
    trace = trace[trace['model'].isin(models)]
    trace = trace.groupby('model').apply(lambda df: df.sort_values(by='duration', ascending=False).iloc[0])[['%cpu', 'peak_rss', 'peak_vmem', 'rchar', 'wchar', 'duration']]


    def convert_duration_to_hours(duration_str):
        import re
        hours, minutes, seconds = 0, 0, 0
        time_parts = re.findall(r'(\d+)([hms])', duration_str)
        for value, unit in time_parts:
            if unit == 'h':
                hours = int(value)
            elif unit == 'm':
                minutes = int(value)
            elif unit == 's':
                seconds = int(value)
        return (hours * 3600 + minutes * 60 + seconds)/3600
    def format_ram(row):
        value = float(row.split()[0])
        if 'GB' in row:
            value = value
        elif 'MB' in row:
            value = value/1000
        else:
            raise ValueError('Define')
        return value 

    for col in trace.columns:
        if col=='%cpu':
            trace[col] = trace[col].str.replace('%', '').astype(float)
        elif col=='duration':
            trace[col] = trace[col].apply(convert_duration_to_hours)
        else:
            trace[col] = trace[col].apply(format_ram)
    return trace 

models = ['pearson_causal', 'pearson_corr', 'positive_control']
base_dir = 'resources/results/d0_hvgs_baseline'
trace = pd.read_csv(f'{base_dir}/trace.txt', sep='\t')

trace_baselines = process_trace(trace)

models = ['celloracle']
base_dir = 'resources/results/celloracle_d0_hvgs'
trace = pd.read_csv(f'{base_dir}/trace.txt', sep='\t')
trace_models = process_trace(trace)


trace_seqera = pd.concat([trace_baselines, trace_models], axis=0)
trace_seqera

/vol/tmp/users/jnourisa/ipykernel_1858346/2779943256.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  trace = trace.groupby('model').apply(lambda df: df.sort_values(by='duration', ascending=False).iloc[0])[['%cpu', 'peak_rss', 'peak_vmem', 'rchar', 'wchar', 'duration']]
/vol/tmp/users/jnourisa/ipykernel_1858346/2779943256.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  trace = trace.groupby('model').

,%cpu,peak_rss,peak_vmem,rchar,wchar,duration
model,,,,,,
pearson_causal,515.0,0.9747,4.0,0.0659,0.0014,0.064167
pearson_corr,528.2,0.9751,4.0,0.0659,0.0014,0.072500
positive_control,200.1,4.9000,8.0,2.3000,0.0018,0.075278
celloracle,799.6,14.9000,35.4,18.0000,86.1000,1.472222


### merge local and cluster dfs 

In [8]:
# merge local and cluster dfs 
map_dict = {'peak_rss':'MaxRSS', 'duration':'Elapsed'}

df_local = df_local[map_dict.values()]
trace_seqera = trace_seqera[map_dict.keys()]

trace_seqera.columns = trace_seqera.columns.map(map_dict)


df_res = pd.concat([trace_seqera, df_local], axis=0)

df_res.columns = ['Peak memory (GB)', 'Duration (hour)']

df_res

,Peak memory (GB),Duration (hour)
pearson_causal,0.974700,0.064167
pearson_corr,0.975100,0.072500
positive_control,4.900000,0.075278
celloracle,14.900000,1.472222
portia,46.943497,0.110556
grnboost2,3.067471,1.568056
scenic,30.356461,1.908056
genie3,13.105103,16.682500
ppcor,3.909119,0.556667
scglue,29.917423,4.380278


## Merge scores with resources

In [41]:
# create ranking 
df_scores = df_scores.fillna(0)
df_scores[df_scores < 0]=0
df_scores = (df_scores-df_scores.min(axis=0))/(df_scores.max(axis=0)-df_scores.min(axis=0))
df_scores['overall_score'] = df_scores.mean(axis=1)
# df_scores['rank'] = df_scores.mean(axis=1).rank(ascending=False).astype(int)

df_all = pd.concat([df_scores, df_res], axis=1)
df_all = df_all.fillna(0)
df_all.index.name = 'method_name' 
df_all = df_all.reset_index()

df_all 


,method_name,ex(False)_tf(-1),ex(True)_tf(-1),static-theta-0.0,static-theta-0.5,overall_score,Peak memory (GB),Duration (hour)
0,collectri,0.000000,0.000000,0.240505,0.095068,0.077646,0.000000,0.000000
1,negative_control,0.000000,0.000000,0.162005,0.000000,0.032401,0.000000,0.000000
2,positive_control,1.000000,1.000000,0.671716,0.668815,0.860974,4.900000,0.075278
3,pearson_corr,0.487919,0.759962,0.344838,0.184772,0.453523,0.975100,0.072500
4,pearson_causal,0.726277,0.854683,0.894789,0.533161,0.774527,0.974700,0.064167
5,portia,0.304491,0.335592,0.141693,0.125352,0.226639,46.943497,0.110556
6,ppcor,0.046705,0.138974,0.000000,0.046814,0.050776,3.909119,0.556667
7,genie3,0.761818,0.724142,0.927879,0.687766,0.798691,13.105103,16.682500
8,grnboost2,0.778974,0.679105,1.000000,1.000000,0.891616,3.067471,1.568056
9,scenic,0.295812,0.305056,0.530045,0.665799,0.458534,30.356461,1.908056


## Summary figure

In [49]:

summary_file = "output/summary_d0_hvg.tsv"
summary_figure = "output/summary_d0_hvg_figure.pdf"

df_all['memory_log'] = np.log(df_all['Peak memory (GB)']+1)
df_all['memory_log'] = np.max(df_all['memory_log'])-df_all['memory_log']


df_all["duration_log"] = np.log(df_all['Duration (hour)']+1)
df_all['duration_log'] = np.max(df_all['duration_log'])-df_all['duration_log']

df_all["duration_str"] = df_all['Duration (hour)'].round(1).astype(str)
df_all['memory_str'] =  df_all['Peak memory (GB)'].round(1).astype(str)


df_all.to_csv(summary_file, sep='\t')

!Rscript ../grn_benchmark/src/metrics_figure.R {summary_file} {summary_figure}


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──
✔ ggplot2 3.5.1     ✔ purrr   0.3.4
✔ tibble  3.2.1     ✔ dplyr   1.1.4
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Warning message:
`thisfile()` was deprecated in rprojroot 2.0.0.
ℹ Please use `whereami::thisfile()` instead. 
New names:
• `` -> `...1`
Rows: 12 Columns: 13
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr  (1): method_name
dbl (12): ...1, ex(False)_tf(-1), ex(True)_tf(-1), static-theta-0.0, static-...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
# A tibble: 10 × 7
   id               id_color         name       group geom  palette options     
   <c

# Metacell

In [50]:
# if par['metacell']:
#     print('metacell')
#     def create_meta_cells(df, n_cells=15):
#         meta_x = []
#         for i in range(0, df.shape[0], n_cells):
#             meta_x.append(df.iloc[i:i+n_cells, :].sum(axis=0).values)
#         df = pd.DataFrame(meta_x, columns=df.columns)
#         return df
            
#     adata_df = pd.DataFrame(multiomics_rna.X.todense(), columns=multiomics_rna.var_names)
#     adata_df['cell_type'] = multiomics_rna.obs['cell_type'].values
#     adata_df['donor_id'] = multiomics_rna.obs['donor_id'].values
#     df = adata_df.groupby(['cell_type','donor_id']).apply(lambda df: create_meta_cells(df))
#     X = df.values
#     var = pd.DataFrame(index=df.columns)
#     obs = df.reset_index()[['cell_type','donor_id']]
#     multiomics_rna = ad.AnnData(X=X, obs=obs, var=var)

# Robustness analysis

In [1]:
# !viash ns build --setup cb --parallel
!viash ns build --setup cb -q run_robustness_analysis


temporaryFolder: /tmp/viash_hub_repo10268443552668753296 uri: https://github.com/openproblems-bio/openproblems-v2.git
Cloning into '.'...
checkout out: List(git, checkout, origin/main_build, --, .) 0 
Exporting run_robustness_analysis (workflows) =nextflow=> /mnt/c/Users/nourisa/Documents/testProjs/ongoing/task_grn_inference/target/nextflow/workflows/run_robustness_analysis
Not all configs built successfully
  39 configs were disabled
  1/1 configs built successfully


## Permute net

In [3]:
!bash scripts/run_robust_analys.sh "net"

In [7]:
!aws s3 sync s3://openproblems-data/resources/grn/results/robust_analy_reg2_net ./resources/results/robust_analy_reg2_net

download: s3://openproblems-data/resources/grn/results/robust_analy_reg2_net/state.yaml to resources/results/robust_analy_reg2_net/state.yaml
download: s3://openproblems-data/resources/grn/results/robust_analy_reg2_net/scores.yaml to resources/results/robust_analy_reg2_net/scores.yaml
download: s3://openproblems-data/resources/grn/results/robust_analy_reg2_net/metric_configs.yaml to resources/results/robust_analy_reg2_net/metric_configs.yaml
download: s3://openproblems-data/resources/grn/results/robust_analy_reg2_net/trace.txt to resources/results/robust_analy_reg2_net/trace.txt


## Add noise to weights

In [41]:
!bash scripts/run_robust_analys.sh "weight"

weight
Nextflow 24.04.4 is available - Please consider updating your version to it
N E X T F L O W  ~  version 23.10.4
Launching `target/nextflow/workflows/run_robustness_analysis/main.nf` [dreamy_davinci] DSL2 - revision: 6bd5cda1f4
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:run... -

[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:pub... -

executor >  local (1)
[5f/aa4675] process > run_robustness_analysis:run... [  0%] 0 of 30
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:pub... -

executor >  local (6)
[f3/689356] process > run_robustness_analysis:run... [  0%] 0 of 30
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustn

In [9]:
!aws s3 sync s3://openproblems-data/resources/grn/results/robust_analy_reg2_weight ./resources/results/robust_analy_reg2_weight

download: s3://openproblems-data/resources/grn/results/robust_analy_reg2_weight/scores.yaml to resources/results/robust_analy_reg2_weight/scores.yaml
download: s3://openproblems-data/resources/grn/results/robust_analy_reg2_weight/state.yaml to resources/results/robust_analy_reg2_weight/state.yaml
download: s3://openproblems-data/resources/grn/results/robust_analy_reg2_weight/metric_configs.yaml to resources/results/robust_analy_reg2_weight/metric_configs.yaml
download: s3://openproblems-data/resources/grn/results/robust_analy_reg2_weight/trace.txt to resources/results/robust_analy_reg2_weight/trace.txt


## Negative vs positive controls

In [8]:
!bash scripts/run_pc_vs_nc.sh "pc_vs_nc"

In [12]:
!aws s3 sync s3://openproblems-data/resources/grn/results/robust_analy_reg2_pc_vs_nc ./resources/results/robust_analy_reg2_pc_vs_nc

## regulatory sign

In [50]:
!bash scripts/run_robust_analys.sh "sign"

sign
Nextflow 24.04.4 is available - Please consider updating your version to it
N E X T F L O W  ~  version 23.10.4
Launching `target/nextflow/workflows/run_robustness_analysis/main.nf` [soggy_sinoussi] DSL2 - revision: d55e1b9116
[-        ] process > run_robustness_analysis:run... -

[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:run... -

[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:pub... -

executor >  local (2)
[2f/3f5479] process > run_robustness_analysis:run... [  0%] 0 of 30
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:pub... -

executor >  local (6)
[f9/652d7d] process > run_robustness_analysis:run... [  0%] 0 of 30
[-        ] process > run_robustne

In [10]:
!aws s3 sync s3://openproblems-data/resources/grn/results/robust_analy_reg2_sign ./resources/results/robust_analy_reg2_sign

download: s3://openproblems-data/resources/grn/results/robust_analy_reg2_sign/trace.txt to resources/results/robust_analy_reg2_sign/trace.txt
download: s3://openproblems-data/resources/grn/results/robust_analy_reg2_sign/state.yaml to resources/results/robust_analy_reg2_sign/state.yaml
download: s3://openproblems-data/resources/grn/results/robust_analy_reg2_sign/scores.yaml to resources/results/robust_analy_reg2_sign/scores.yaml
download: s3://openproblems-data/resources/grn/results/robust_analy_reg2_sign/metric_configs.yaml to resources/results/robust_analy_reg2_sign/metric_configs.yaml


## Corr vs causal

In [ ]:
!bash scripts/run_robust_analys_causal.sh 

In [4]:
!aws s3 sync s3://openproblems-data/resources/grn/results/robust_analy_causal_1 ./resources/results/robust_analy_causal_1

download: s3://openproblems-data/resources/grn/results/robust_analy_causal_1/scores.yaml to resources/results/robust_analy_causal_1/scores.yaml
download: s3://openproblems-data/resources/grn/results/robust_analy_causal_1/state.yaml to resources/results/robust_analy_causal_1/state.yaml
download: s3://openproblems-data/resources/grn/results/robust_analy_causal_1/trace.txt to resources/results/robust_analy_causal_1/trace.txt
download: s3://openproblems-data/resources/grn/results/robust_analy_causal_1/metric_configs.yaml to resources/results/robust_analy_causal_1/metric_configs.yaml


In [13]:
!viash run src/robustness_analysis/causal/config.vsh.yaml -- --causal true 
!viash run src/metrics/regression_1/config.vsh.yaml -- --prediction output/prediction.csv

Read data

Noramlize data
Create corr net
Processing groups: 100%|██████████| 4/4 [00:41<00:00, 10.47s/it]
Output GRN


In [56]:
# 
corr_scores = []
for i in range(1, n_iter):    
    corr_scores.append(ad.read_h5ad(f"output/causal/scores/net_corr_{i}.h5ad").uns['metric_values'][0])
pc_score = ad.read_h5ad(f"output/causal/scores/net_pc.h5ad").uns['metric_values'][0]

In [59]:
(corr_scores<pc_score).sum()

95